In [1]:
import json
import pandas as pd
import re
import ast

C:\Users\met48\AppData\Local\Temp\ipykernel_24300\2245749428.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# PlumX Metrics

In [2]:
f = open('plumx_metrics_data.json')
data_json = json.load(f)

In [3]:
len(data_json)

24521

In [4]:
data_flat = pd.DataFrame()
for item in data_json:
    item_flat =  pd.json_normalize(item)
    data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')

In [5]:
data_flat.shape

(24521, 20)

In [6]:
data_flat.to_csv("plumx_metrics_tabular_uncleaned.csv", index=False)

# Altmetrics

In [7]:
f = open('altmetric_data.json')
altmetrics_corpus = f.read()
altmetrics_corpus = altmetrics_corpus.replace('\\', '')
altmetrics_corpus = altmetrics_corpus.replace(',', ', ')
altmetrics_corpus = altmetrics_corpus.replace(':', ': ')
altmetrics_corpus = altmetrics_corpus[1:-1]

In [8]:
index = 0
item_start_index = None
item_end_index = None
total = len(altmetrics_corpus)
stack = 0
item_indices = list()
while(index < total):
    if(altmetrics_corpus[index]=='{'):
        if(stack == 0): # opening new item 
            item_start_index = index
            stack+=1
            index+=1 
        else: # nested data within item 
            stack+=1
            index+=1
    elif(altmetrics_corpus[index]=='}'):
        if(stack == 1): # closing item
            item_end_index = index+1
            item_indices.append((item_start_index, item_end_index))
            stack-=1
            index+=1
        else: # closing nested data within item
            stack-=1
            index+=1
    else:
        index+=1   

In [9]:
len(item_indices) # should be 8416

8436

In [10]:
data_flat = pd.DataFrame()
error_count = 0
for item in item_indices:
    text = altmetrics_corpus[item[0]:item[1]]
    try:
        item_dict = json.loads(text)
        item_flat =  pd.json_normalize(item_dict)
        data_flat = pd.concat([data_flat, item_flat], axis=0, join='outer')
    except:
        print("Unable to parse start index ", item[0], " end index ", item[1], " .")
        error_count +=1
print("Error count: ", error_count)

Unable to parse start index  122038  end index  126578  .
Unable to parse start index  183537  end index  186779  .
Unable to parse start index  293093  end index  296296  .
Unable to parse start index  321897  end index  326099  .
Unable to parse start index  737328  end index  740263  .
Unable to parse start index  1020341  end index  1024523  .
Unable to parse start index  1117830  end index  1122690  .
Unable to parse start index  1179242  end index  1183834  .
Unable to parse start index  1307301  end index  1310847  .
Unable to parse start index  1710827  end index  1714870  .
Unable to parse start index  1752149  end index  1755842  .
Unable to parse start index  1820636  end index  1824162  .
Unable to parse start index  1835807  end index  1839204  .
Unable to parse start index  2075248  end index  2084079  .
Unable to parse start index  2764540  end index  2769887  .
Unable to parse start index  3085783  end index  3088763  .
Unable to parse start index  3303248  end index  3

We will ignore the parsing errors for now.  They appear to be caused by the use of quotation marks in the abstract and 110 (1.3%) samples are affected. This will be addressed in a later update.

In [11]:
data_flat.shape

(8327, 100)

In [12]:
data_flat.to_csv("altmetrics_tabular_uncleaned.csv", index=False)